In [58]:
import csv
import pandas as pd
import itertools
import math
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
import string
import re
import nltk
from google.colab import files
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import classification_report




In [5]:
uploaded = files.upload()

news = pd.read_csv('news.csv')

Saving news.csv to news (3).csv


In [6]:
news.head()

,id,title,score,subreddit,url,num_comments,text,created,police_brutality
0,dfn3yi,Blizzard Employees Staged a Walkout After the ...,226324,news,https://www.thedailybeast.com/blizzard-employe...,9623,NaN,1.570683e+09,0
1,eubjfc,Kobe Bryant killed in helicopter crash in Cali...,213684,news,https://www.fox5dc.com/news/kobe-bryant-killed...,20705,NaN,1.580096e+09,0
2,84aebi,Scientist Stephen Hawking has died aged 76,188175,news,http://news.sky.com/story/scientist-stephen-ha...,6916,NaN,1.521028e+09,0
3,dp5lr1,Jeffrey Epstein's autopsy more consistent with...,186243,news,https://www.foxnews.com/us/forensic-pathologis...,10057,NaN,1.572465e+09,0
4,7ej943,F.C.C. Announces Plan to Repeal Net Neutrality,178001,news,https://www.nytimes.com/2017/11/21/technology/...,10857,NaN,1.511312e+09,0


In [7]:
news=news.drop(['id', 'score','subreddit', 'url','num_comments','text','created'], axis=1)

In [8]:
news.head()

,title,police_brutality
0,Blizzard Employees Staged a Walkout After the ...,0
1,Kobe Bryant killed in helicopter crash in Cali...,0
2,Scientist Stephen Hawking has died aged 76,0
3,Jeffrey Epstein's autopsy more consistent with...,0
4,F.C.C. Announces Plan to Repeal Net Neutrality,0


In [11]:
uploaded = files.upload()
police_use_force = pd.read_csv('police_use_force.csv')

Saving police_use_force.csv to police_use_force.csv


In [12]:
police_use_force['title']=police_use_force['name']

In [19]:
police_use_force=police_use_force.drop(['name'], axis=1)

In [20]:
police_use_force.head()

,police_brutality,title
0,1,Compilation of incidents
1,1,Police shove a protesting woman down a set of ...
2,1,Police throw tear-gas at protesters on a bridge.
3,1,Police assault protesters
4,1,Police shoot non-violent protester in the head


In [27]:
frames = [news, police_use_force]

In [28]:
df=pd.concat(frames)

In [29]:
df.shape

(1988, 2)

In [30]:
df.head()

,title,police_brutality
0,Blizzard Employees Staged a Walkout After the ...,0
1,Kobe Bryant killed in helicopter crash in Cali...,0
2,Scientist Stephen Hawking has died aged 76,0
3,Jeffrey Epstein's autopsy more consistent with...,0
4,F.C.C. Announces Plan to Repeal Net Neutrality,0


In [32]:
df.shape

(1988, 2)

In [33]:
nlp = spacy.load("en")

In [34]:
def tokenize(text):
    text = nlp(text)
    tokens = [token.lemma_ for token in text if (token.is_stop != True) and (token.is_punct != True) and (token.text != " ")]
    return tokens

def retoken(text):   
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', text)
    tokens = tokens.lower().split()
    tokens = str(tokens)
    
    return tokens

In [35]:
df['tokens'] = df['title'].apply(tokenize)

In [36]:
df['tokens'] = df['title'].apply(retoken)

In [37]:
df.head()

,title,police_brutality,tokens
0,Blizzard Employees Staged a Walkout After the ...,0,"['blizzard', 'employees', 'staged', 'a', 'walk..."
1,Kobe Bryant killed in helicopter crash in Cali...,0,"['kobe', 'bryant', 'killed', 'in', 'helicopter..."
2,Scientist Stephen Hawking has died aged 76,0,"['scientist', 'stephen', 'hawking', 'has', 'di..."
3,Jeffrey Epstein's autopsy more consistent with...,0,"['jeffrey', 'epsteins', 'autopsy', 'more', 'co..."
4,F.C.C. Announces Plan to Repeal Net Neutrality,0,"['fcc', 'announces', 'plan', 'to', 'repeal', '..."


In [39]:
vect = CountVectorizer()
vect.fit(df['tokens'])
dtm = vect.transform(df['tokens'])

In [40]:
dtm_df = pd.DataFrame(dtm.todense(), columns = vect.get_feature_names())

In [41]:
dtm_df.head()

,10,100,1000,10000,100000,100150,101,104,107,10acre,10bn,10th,10year,10yearold,11,11000,112,117,1191,11943,11th,11yearold,12,1200,120000,127700,12m,12th,12yearold,12yearolds,13,130,1300,13k,13th,14,14000,141,14yearold,15,...,wrongful,wrote,wtop,wuhan,wv,xbox,xi,xl,xxx,yang,yard,yards,yates,year,years,yearslong,yeezy,yelled,yelling,yellow,yells,yellsmerry,yiannopoulos,york,you,youd,young,younger,youporn,your,youre,youtube,zap,zealand,zeroes,zimbabwe,zip,zoo,zoom,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
X = df['title']
y = df['police_brutality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1590,)
(398,)
(1590,)
(398,)


In [52]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier()),
])

In [53]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [56]:
predictions = pipeline.predict(X_test)

In [59]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94       208
           1       0.90      0.98      0.94       190

    accuracy                           0.94       398
   macro avg       0.94      0.94      0.94       398
weighted avg       0.95      0.94      0.94       398

